## IMPORTS

In [22]:
import cv2
cv2.setLogLevel(0)
import numpy as np
import socket
import queue
import threading
import time
import torch
import psutil
import warnings

battery = psutil.sensors_battery()
# torch.set_num_threads(20)
# device = "mps" if torch.backends.mps.is_available() else "cpu"

## CUSTOM CLASS

In [3]:
class FPS:
    def __init__(self):
        self.start_time = time.time()
        self.new_latency = 0
        self.prev_latency = 0
        self.avg_latency = 0
        self.num_frames = 0
        self.avg_fps = 0
        self.fps_threshold = 1
        self.text = ""
    
    def start(self):
        self.new_latency = time.time()
    
    def update(self):
        
        self.num_frames += 1
        
        # perform FPS calculations
        if (time.time() - self.start_time) > self.fps_threshold:
            self.avg_latency = round(1000*(self.new_latency - self.prev_latency))
            self.avg_fps = int(self.num_frames/(time.time() - self.start_time))
            self.text = "FPS: " + str(self.avg_fps) + "  Latency: " + str(self.avg_latency) + " ms"
            self.num_frames = 0
            self.start_time = time.time()
        
        self.prev_latency = self.new_latency
            

## HELPER METHODS

In [2]:
def receive_detections(run_receive_thread,sock,detections_queue):

    while (run_receive_thread):
        detection, addr = sock.recvfrom(65535)
        detections_queue.put(detection)
    
    return()

### LOCAL YOLO MODEL

In [37]:
def run_local_model():
    # import small YOLO model
    model = torch.hub.load('Ultralytics/yolov5', 'yolov5s').to("cpu")

    # Set Video Properties
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    text_font = cv2.FONT_HERSHEY_SIMPLEX
    white = (255,255,255)
    red = (0,0,255)
    black = (0,0,0)
    fps = FPS()

    confidence_fps_count = 0
    avg_fps = 0

    
    print("\n\n***********************")
    print("\nRUNNING LOCAL MODEL...")
    print("\n***********************")
    while True:
        # Capture frame from webcam, get predictions
        fps.start()
        ret, frame = cap.read()
        results = model(frame)
        results = np.array(results.pandas().xyxy[0])
        temp_img = np.copy(frame)
        
        if(len(results) > 0):
            # draw text and bounding boxes for each detection
            for box in results:
                x1,y1 = int(box[0]),int(box[1])
                x2,y2 = int(box[2]),int(box[3])
                rounded_confidence = round(box[4], 2)
                prediction_text = box[6] + "  "+ str(rounded_confidence)
                (text_w, _), _ = cv2.getTextSize(prediction_text, text_font, .6, 1)
                cv2.rectangle(temp_img, (x1,y1), (x2,y2),red, 3)
                cv2.rectangle(temp_img, (x1, y1 - 25), (x1 + text_w, y1), red, -1)
                cv2.putText(temp_img, prediction_text,(x1 ,y1 - 10), text_font, .6, white, 1, cv2.LINE_AA)

            confidence_fps_count += 1
            avg_fps += fps.avg_fps
            avg_fps /= 2
        
        # draw text and background for FPS and Latency
        cv2.rectangle(temp_img, (0,0),(220, 32), black, -1)
        cv2.putText(temp_img, fps.text, (7,20), text_font, .5, white, 1, cv2.LINE_AA)           
        cv2.imshow("Local Processing",temp_img)

        fps.update()
        battery = psutil.sensors_battery()
        
        
        ## *********************************** ##
        #  Check if we should quit local model  #
        ## *********************************** ##
        
        if((battery.percent < 80) and (not battery.power_plugged)):
            print("\n\nBATTERY UNPLUGGED AND BELOW THRESHOLD, QUITTING LOCAL MODEL")
            print("Current battery:",battery.percent)
            break
        
        if(confidence_fps_count >= 300):
            if(avg_fps < 10):
                print("\n\nFPS TOO LOW, QUITTING LOCAL MODEL")
                break
            
            confidence_fps_count = 0
            avg_fps = 0

        # Wait for 1 millisecond, and check if the user pressed the 'q' key or the window was closed
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
    
    # Graceful Shutdown
    cap.release()
    cv2.destroyAllWindows()

### REMOTE PROCESSING ON SERVER

In [20]:
def run_remote_model():
    # Define IP address and port number of the client, setup socket
    ip = '192.168.1.189'
    port = 5005
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

    # setup receiving thread
    detections_queue = queue.Queue()
    run_receive_thread = True
    t1 = threading.Thread(target=receive_detections, args=(run_receive_thread,sock,detections_queue,))
    t1.start()

    # Create video capture object
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 70]

    text_font = cv2.FONT_HERSHEY_SIMPLEX
    white = (255,255,255)
    red = (0,0,255)
    black = (0,0,0)
    
    fps = FPS()
    confidence_fps_count = 0
    avg_fps = 0
    
    
    print("\n\n***********************")
    print("\nRUNNING REMOTE MODEL...")
    print("\n***********************")
    
    while True:
        # Capture frame from webcam, keep track of fps
        fps.start()
        ret, frame = cap.read()

        # Encode frame as JPEG, convert to bytes
        _, img_encoded = cv2.imencode('.jpg', frame, encode_param)
        data = np.array(img_encoded)
        string_data = data.tobytes()

        # Send frame to client
        try:
            sock.sendto(string_data, (ip, port))
        except:
            print("ERROR - FRAME NOT SENT")

        # render frames with FPS + latency overlay
        while not detections_queue.empty():
            detection = detections_queue.get()
            img_encoded = np.frombuffer(detection, dtype=np.uint8)
            img = cv2.imdecode(img_encoded, cv2.IMREAD_COLOR)[:,:,::-1]
            temp_img = np.copy(img)

            # draw text and background for FPS and Latency
            cv2.rectangle(temp_img, (0,0),(220, 32), black, -1)
            cv2.putText(temp_img, fps.text, (7,20), text_font, .5, white, 1, cv2.LINE_AA)             
            cv2.imshow("Remote Processing",temp_img)

            fps.update()
            
            confidence_fps_count += 1
            avg_fps += fps.avg_fps
            avg_fps /= 2
        
        
        ## ************************************ ##
        #  Check if we should quit remote model  #
        ## ************************************ ##
        if(confidence_fps_count >= 50 ):
            if(fps.avg_latency > 300):
                print("\n\nLATENCY TOO HIGH, QUITTING REMOTE MODEL ")
                print("Average Latency:", fps.avg_latency, "ms")
                break
            if(avg_fps < 10):
                print("\n\nFPS TOO LOW, QUITTING REMOTE MODEL")
                print("Average FPS:", avg_fps)
                break
            
            confidence_fps_count = 0
            avg_fps = 0

        
        # Wait for 1 millisecond, and check if the user pressed 'q'
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
    
    # Graceful shutdown 
    cap.release()
    cv2.destroyAllWindows()
    run_receive_thread = False

# DEEP DECISION

In [38]:
# run indefinitely
while True:
    run_local_model()
    run_remote_model()

Using cache found in /Users/mauro/.cache/torch/hub/Ultralytics_yolov5_master
YOLOv5 🚀 2023-4-18 Python-3.9.16 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


requirements: /Users/mauro/.cache/torch/hub/requirements.txt not found, check failed.


***********************

RUNNING LOCAL MODEL...

***********************


BATTERY UNPLUGGED AND BELOW THRESHOLD, QUITTING LOCAL MODEL
Current battery: 73


***********************

RUNNING REMOTE MODEL...

***********************


Using cache found in /Users/mauro/.cache/torch/hub/Ultralytics_yolov5_master
YOLOv5 🚀 2023-4-18 Python-3.9.16 torch-2.0.0 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


requirements: /Users/mauro/.cache/torch/hub/requirements.txt not found, check failed.


***********************

RUNNING LOCAL MODEL...

***********************


BATTERY UNPLUGGED AND BELOW THRESHOLD, QUITTING LOCAL MODEL
Current battery: 73


***********************

RUNNING REMOTE MODEL...

***********************


KeyboardInterrupt: 